In [2]:
from io import StringIO
from pathlib import Path

from dapla_pseudo import PseudoData
import pandas as pd

JSON_FILE = "data/personer.json"
CSV_FILE = "data/personer.csv"

df = pd.read_json(
    JSON_FILE,
    dtype={
        "fnr": "string",
        "fornavn": "string",
        "etternavn": "string",
        "kjonn": "category",
        "fodselsdato": "string",
    },
)
df.head()

,fnr,fornavn,etternavn,kjonn,fodselsdato
0,11854898347,Donald,Duck,M,020995
1,01839899544,Mikke,Mus,M,060970
2,16910599481,Anton,Duck,M,180999


# Case: Single field default encryption (daead)

In [3]:
result = (
    PseudoData.from_pandas(df).on_field("fnr").pseudonymize()
)
result.to_polars().head()

KeyError: 'LOCAL_USER_PATH'

# Case: Single field sid mapping

In [ ]:
result = (
    PseudoData.from_pandas(df)
    .on_field("fnr")
    .map_to_stable_id()
    .pseudonymize()
)
result.to_polars().head()

# Case: Single field FPE (used for e.g. existing stable ID/snr/sid)

In [ ]:
result = (
    PseudoData.from_pandas(df)
    .on_field("fnr")
    .pseudonymize(preserve_formatting=True)
)
result.to_polars().head()

# Case: Multiple fields default encryption (daead)

In [ ]:
result = (
    PseudoData.from_pandas(df)
    .on_fields("fornavn", "etternavn", "fodselsdato")
    .pseudonymize()
)
result.to_polars().head()

# Case: Chaining calls
Calls may simply be chained together to apply different pseudonymization to different fields.

In [ ]:
result = (
    PseudoData.from_pandas(df)
    .on_field("fnr")
    .map_to_stable_id()
    .pseudonymize()
)
result = (
    PseudoData.from_pandas(result.to_polars())
    .on_fields("fornavn", "etternavn", "fodselsdato")
    .pseudonymize()
)
result.to_polars().head()